#  Hoeffding Inequality

Run a computer simulation for flipping 1,000 virtual fair coins. Flip each coin independently 10 times. Focus on 3 coins as follows: c1 is the first coin flipped, crand is a coin chosen randomly from the 1,000, and cmin is the coin which had the minimum frequency of heads (pick the earlier one in case of a tie). Let ν1, νrand, and νmin be the fraction of heads obtained for the 3 respective coins out of the 10 tosses. Run the experiment 100,000 times in order to get a full distribution of ν1, νrand, and νmin (note that crand and cmin will change from run to run).  
1. The average value of νmin is closest to:  
[a] 0 **[b] 0.01** [c] 0.1 [d] 0.5 [e] 0.67  
2. Which coin(s) has a distribution of ν that satisfies the (single-bin) Hoeffding Inequality?  
[a] c1 only [b] crand only [c] cmin only **[d] c1 and crand**
[e] cmin and cran  

In [1]:
import random

In [2]:
def experiment(times):
    v_1 = 0
    v_rand = 0
    v_min = 0
    for t in range(0,times):
        head_freq = [0]*1000
        c_1 = 0
        c_rand = random.randint(0,999)
        for i in range(0,1000):
            for _ in range(0,10):
                head_freq[i] += random.randint(0,1)
        c_min = head_freq.index(min(head_freq))
        v_1 += head_freq[c_1]
        v_rand += head_freq[c_rand]
        v_min += head_freq[c_min]
    return v_1/(times*10.0),v_rand/(times*10.0),v_min/(times*10.0)

In [3]:
v_1,v_rand,v_min = experiment(100000)
print "v_1=",v_1,"v_rand=",v_rand,"v_min=",v_min

v_1= 0.499852 v_rand= 0.499968 v_min= 0.037697


# Linear Regression

In these problems, we will explore how Linear Regression for classification works. As with the Perceptron Learning Algorithm in Homework # 1, you will create your own target function f and data set D. Take d = 2 so you can visualize the problem, and assume X = [−1, 1] × [−1, 1] with uniform probability of picking each x ∈ X. In each run, choose a random line in the plane as your target function f (do this by taking two random, uniformly distributed points in [−1, 1] × [−1, 1] and taking the line passing through them), where one side of the line maps to +1 and the other maps to −1. Choose the inputs xn of the data set as random points (uniformly in X), and evaluate the target function on each xn to get the corresponding output yn.  

In [4]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [5]:
WLIN = np.array([0,0,0],dtype=np.float)
WF = np.array([0,0,-1],dtype=np.float)

In [6]:
def line(x,w):
    return (-w[0]-w[1]*x)/w[2]

In [7]:
def draw(data,label,):
    idx_1 = np.where(label==1)
    p1 = plt.scatter(data[idx_1,1],data[idx_1,2],marker='o',color = 'g',label = '1',s = 30)

    idx_2 = np.where(label==-1)
    p2 = plt.scatter(data[idx_2,1],data[idx_2,2],marker='x',color = 'r',label = '-1',s = 30)

    p_WLIN = plt.plot([-1,1],line(np.array([-1,1]),WLIN),'b-',linewidth=2,label="WLIN")
    p_WF = plt.plot([-1,1],line(np.array([-1,1]),WF),'r-.',linewidth=2,label="WF")
    plt.legend(loc='upper right')# add legend,need to set label first
    plt.show()

In [8]:
def slove_LRegress(X,y):
    global WLIN
    A = np.dot(X.T,X)
    pseudo_inverse = np.dot(np.linalg.inv(A),X.T)
    WLIN = np.dot(pseudo_inverse,y)

In [9]:
def Linear_Regression(N,noise=False,pic = False):
    global WF
    data = np.random.uniform(-1,1,(N,2))
    data = np.insert(data,0,values=np.ones(N),axis=1)
    point = np.random.uniform(-1,1,(2,2))
    k = (point[0][1] - point[1][1])/(point[0][0] - point[1][0])
    b = point[0][1] - k*point[0][0]
    WF = np.array([b,k,-1],dtype=np.float)
    label = np.dot(data,WF)
    label[label>0]=1
    label[label<=0]=-1
    if noise:
        noise_point = random.sample(range(0,N),int(N*0.1))
        for i in noise_point:
            label[i] =-label[i]
    slove_LRegress(data,label)
    if pic:
        draw(data,label)
    error = 0
    pred = np.dot(data,WLIN)
    pred[pred>0]=1
    pred[pred<=0]=-1
    error = sum(pred!=label)
    return error/float(N)

5.Take N = 100. Use Linear Regression to find g and evaluate Ein, the fraction of in-sample points which got classified incorrectly. Repeat the experiment 1000 times and take the average (keep the g’s as they will be used again in Problem 6). Which of the following values is closest to the average Ein? (Closest is the option that makes the expression |your answer−given option| closest to 0. Use this definition of closest here and throughout.)  
[a] 0 [b] 0.001 **[c] 0.01** [d] 0.1
[e] 0.5  

In [10]:
def compute_Ein(N,times= 1000,noise = False):
    E_in = 0
    for _ in range(times):
        E_in += Linear_Regression(N,noise,pic = False,)
    return E_in/float(times)

In [11]:
print "E_in=",compute_Ein(100)

E_in= 0.04029


6.Now, generate 1000 fresh points and use them to estimate the out-of-sample error Eout of g that you got in Problem 5 (number of misclassified out-of-sample points / total number of out-of-sample points). Again, run the experiment 1000 times and take the average. Which value is closest to the average Eout?  
[a] 0 [b] 0.001 **[c] 0.01** [d] 0.1
[e] 0.5  

In [12]:
def compute_Eout(N,times = 1000):
    E_out = 0
    for _ in range(times):
        data = np.random.uniform(-1,1,(N,2))
        data = np.insert(data,0,values=np.ones(N),axis=1)
        label = np.dot(data,WF)
        label[label>0]=1
        label[label<=0]=-1
        pred = np.dot(data,WLIN)
        pred[pred>0]=1
        pred[pred<=0]=-1
        error = sum(pred!=label)
        E_out += error/float(N)
    return E_out/times

In [13]:
print "E_out=",compute_Eout(1000)

E_out= 0.036851


7.Now, take N = 10. After finding the weights using Linear Regression, use them as a vector of initial weights for the Perceptron Learning Algorithm. Run PLA until it converges to a final vector of weights that completely separates all the in-sample points. Among the choices below, what is the closest value to the average number of iterations (over 1000 runs) that PLA takes to converge? (When implementing PLA, have the algorithm choose a point randomly from the set of misclassified points at each iteration)  
**[a] 1** [b] 15 [c] 300 [d] 5000
[e] 10000  

In [14]:
def PLA(data,label):
    WPLA = WLIN
    #初始化WPLA
    numofdata = data.shape[0]
    t = 0
    while True:
        pred = np.dot(data,WPLA)
        pred[pred>0]=1
        pred[pred<=0]=-1
        error_point = np.where(pred!=label)[0]
        if error_point.size == 0:
            return t
        fix_point = random.sample(error_point,1)[0]#从分类错误的点集中随机选取一个进行修正
        t += 1
        WPLA += label[fix_point]*data[fix_point]

In [15]:
def num_iterations(N,times=1000):
    num_iter = 0
    for _ in range(times):
        data = np.random.uniform(-1, 1, (N, 2))
        data = np.insert(data,0,values=np.ones(N),axis=1)
        #随机生成10个长度为3的数组
        label = np.dot(data,WF)
        label[label>0]=1
        label[label<=0]=-1
        num_iter+= PLA(data,label)    
    return num_iter/float(times)

In [16]:
print "num_iterations=",num_iterations(10,1000)

num_iterations= 0.35


#  Nonlinear Transformation

In these problems, we again apply Linear Regression for classification. Consider the target function:
f(x1, x2) = sign(x2 1 + x2 2 − 0.6)
Generate a training set of N = 1000 points on X = [−1, 1] × [−1, 1] with a uniform probability of picking each x ∈ X. Generate simulated noise by flipping the sign of
the output in a randomly selected 10% subset of the generated training set.

8.Carry out Linear Regression without transformation, i.e., with feature vector: (1, x1, x2),to find the weight w. What is the closest value to the classification in-sample error Ein? (Run the experiment 1000 times and take the average Ein to reduce variation in your results.)  
[a] 0 [b] 0.1 [c] 0.3 **[d] 0.5**
[e] 0.8  

In [17]:
def target_f(x):
    if x[1]**2 + x[2]**2 -0.6 > 0:
        return 1
    else:
        return -1

In [18]:
def NonLinear_Regression(N,pic = False):
    data = np.random.uniform(-1,1,(N,2))
    data = np.insert(data,0,values=np.ones(N),axis=1)
    label = np.apply_along_axis(target_f,1,data)
    noise_point = random.sample(range(0,N),int(N*0.1))
    label.flat[noise_point]=-label.flat[noise_point]
    slove_LRegress(data,label)
    if pic:
        draw(data,label)
    pred = np.dot(data,WLIN)
    pred[pred>0] =1
    pred[pred<=0] = -1
    error = sum(pred!=label)
    return error/float(N)

In [19]:
def compute_Nonlinear_Ein(N,times= 1000):
    E_in = 0
    E_in = sum(map(NonLinear_Regression,[N]*times))
    return E_in/float(times)

In [20]:
print "E_in with noise=",compute_Nonlinear_Ein(1000)

E_in with noise= 0.50303


9.Now, transform the N = 1000 training data into the following nonlinear feature vector:
(1, x1, x2, x1x2, x2 1, x2 2)
Find the vector ˜w that corresponds to the solution of Linear Regression. Which of the following hypotheses is closest to the one you find? Closest here means agrees the most with your hypothesis (has the highest probability of agreeing on a randomly selected point). Average over a few runs to make sure your answer is stable.  
**[a] g(x1, x2) = sign(−1 − 0.05x1 + 0.08x2 + 0.13x1x2 + 1.5x2**  
[b] g(x1, x2) = sign(−1 − 0.05x1 + 0.08x2 + 0.13x1x2 + 1.5x2  
[c] g(x1, x2) = sign(−1 − 0.05x1 + 0.08x2 + 0.13x1x2 + 15x2  
[d] g(x1, x2) = sign(−1 − 1.5x1 + 0.08x2 + 0.13x1x2 + 0.05x2  
[e] g(x1, x2) = sign(−1 − 0.05x1 + 0.08x2 + 1.5x1x2 + 0.15x2
1 + 1.5x2 1 + 15x2 1 + 1.5x2
2)
2) 2)
1 + 0.05x2 1 + 0.15x2
2)
2)

In [21]:
WT = np.zeros(6)

In [22]:
def slove_LRegress_T(X,y):
    global WT
    temp = np.dot(X.T,X)
    pseudo_inverse = np.dot(np.linalg.inv(temp),X.T)
    WT = np.dot(pseudo_inverse,y)

In [23]:
def NonLinear_Regression_T(N,pic = False):
    data = np.random.uniform(-1,1,(N,2))
    data = np.insert(data,0,values=np.ones(N),axis=1)
    data = np.insert(data,3,values=data[:,1]*data[:,2],axis=1)
    data = np.insert(data,4,values=data[:,1]**2,axis=1)
    data = np.insert(data,5,values=data[:,2]**2,axis=1)
    label = np.apply_along_axis(target_f,1,data)
    noise_point = random.sample(range(0,N),int(N*0.1))
    label.flat[noise_point]=-label.flat[noise_point]
    slove_LRegress_T(data,label)
    if pic:
        draw(data,label)

In [24]:
def check_closet(N):
    H = np.array([[-1,-0.05,0.08,0.13,1.5,1.5],
                  [-1,-0.05,0.08,0.13,1.5,15],
                 [-1,-0.05,0.08,0.13,15,1.5],
                 [-1,-1.5,0.08,0.13,0.05,0.05],
                 [-1,-0.05,0.08,1.5,0.15,0.15]])
    data = np.random.uniform(-1,1,(N,2))
    data = np.insert(data,0,values=np.ones(N),axis=1)
    data = np.insert(data,3,values=data[:,1]*data[:,2],axis=1)
    data = np.insert(data,4,values=data[:,1]**2,axis=1)
    data = np.insert(data,5,values=data[:,2]**2,axis=1)
    highest = 0
    ans = 0
    for g in H:
        pred_w = np.dot(data,WT)
        pred_w[pred_w>0] = 1
        pred_w[pred_w<=0] = -1
        pred_g = np.dot(data,g)
        pred_g[pred_g>0] = 1
        pred_g[pred_g<=0] = -1
        correct = sum(pred_w==pred_g)
        if correct > highest:
            highest = correct
            ans = g
    return ans

In [25]:
check_closet(100)

array([-1.  , -0.05,  0.08,  1.5 ,  0.15,  0.15])

10.What is the closest value to the classification out-of-sample error Eout of your hypothesis from Problem 9? (Estimate it by generating a new set of 1000 points and adding noise, as before. Average over 1000 runs to reduce the variation in your results.)  
[a] 0 **[b] 0.1** [c] 0.3 [d] 0.5
[e] 0.8  

In [26]:
def compute_Nonlinear_Eout(N,times = 1000):
    E_out = 0
    for _ in range(times):
        data = np.random.uniform(-1,1,(N,2))
        data = np.insert(data,0,values=np.ones(N),axis=1)
        data = np.insert(data,3,values=data[:,1]*data[:,2],axis=1)
        data = np.insert(data,4,values=data[:,1]**2,axis=1)
        data = np.insert(data,5,values=data[:,2]**2,axis=1)
        label = np.apply_along_axis(target_f,1,data)
        pred = np.dot(data,np.array([-1.  , -0.05,  0.08,  0.13,  1.5 ,  1.5 ]))
        pred[pred>0]=1
        pred[pred<=0]=-1
        error = sum(pred!=label)
        E_out += error/float(N)
    return E_out/times

In [27]:
print "E_out=",compute_Nonlinear_Eout(1000)

E_out= 0.053203
